<a href="https://colab.research.google.com/github/RajKumarPandey/qml_notebooks/blob/main/qiskit_pki_qml_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit qiskit-machine-learning

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import StatevectorSampler
from qiskit.providers.basic_provider import BasicProvider
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms import QSVC
import requests


csv_url = "https://raw.githubusercontent.com/RajKumarPandey/qml_notebooks/main/synthetic_pki_dataset.csv"
response = requests.get(csv_url)
with open("synthetic_pki_dataset.csv", "wb") as f:
    f.write(response.content)

df = pd.read_csv("./synthetic_pki_dataset.csv")
X = df.drop("label", axis=1).values
y = df["label"].values
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

feature_map = ZZFeatureMap(feature_dimension=X_train.shape[1], reps=2)

sampler = StatevectorSampler()


kernel = FidelityQuantumKernel(feature_map=feature_map)


qsvc = QSVC(quantum_kernel=kernel)
qsvc.fit(X_train, y_train)

y_pred = qsvc.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))